In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.com/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content, features="lxml")
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('0')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

df = pd.read_csv("amazon_products.csv")

In [6]:
df

,Book Name,Author,Rating,Customers_Rated,Price
0,Midnight in Washington: How We Almost Lost Our...,Adam Schiff,0,0,$25.49
1,State of Terror: A Novel,Louise Penny,4.0 out of 5 stars,1,$20.99
2,The Storyteller: Tales of Life and Music,Dave Grohl,4.8 out of 5 stars,109,$19.14
3,Paint by Sticker Kids: Christmas: Create 10 Pi...,Workman Publishing,4.8 out of 5 stars,"3,480",$4.78
4,Little Blue Truck's Halloween,Alice Schertle,4.9 out of 5 stars,"7,407",$7.86
...,...,...,...,...,...
95,Junie B. Jones's First Boxed Set Ever! (Books ...,Barbara Park,4.8 out of 5 stars,"12,685",$14.81
96,On Tyranny: Twenty Lessons from the Twentieth ...,Timothy Snyder,4.7 out of 5 stars,"17,152",$5.42
97,Where the Crawdads Sing,Delia Owens,4.8 out of 5 stars,"191,597",$9.98
98,Women Food and Hormones A 4 Week Plan to Achie...,Sara Gottfried,4.0 out of 5 stars,111,$23.49
